<img style="margin-right:80px; margin-left:80px" src=img/DDUST__Nero.png width="170"> <img style="margin-right:80px" src=img/01_Polimi_centrato_BN_positivo.png width="170"> <img style="margin-right:80px" src=img/sigillo_testo_colori_300dpi.png width="170"> <img style="margin-right:80px" src=img/fondazione-politecnico-di-milano.png width="170"> <img src=img/ML_FCARIPLO_cmyk__base_100mm.png width="170">
<br>

----


# API Request notebook for climate and air quality data from Copernicus Atmosphere Monitoring Service (CAMS) and Copernicus Climate Change Service (C3S)

This notebook is used to download data derived from meteorological and air quality models. It is divided into 2 parts:
- The first part requests data using the `CDS API` Python library. It's used to download data concerning air quality variables from Copernicus Atmosphere Monitoring Service (CAMS) models.
- The second part requests data using the `Google Earth Engine` Python API library. It's used to download data concerning meteorological variables obtained from the Copernicus Climate Change Service ERA5-land hourly Reanalysis model.

### Link to Copernicus Services:
- Link to [ADS Atmosphere Copernicus (CAMS)](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)
- Link to [CDS Climate Copernicus (C3S)](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=overview)

### How to access the CDS API:
It's possible to change `key` and `value` pairs provided from the website in the `.cdsapirc` file as explained in the following link: https://ads.atmosphere.copernicus.eu/api-how-to 

Alternatively, `key` and `value` can be used as input directly inside the `cdsapi` API request if the notebook is not shared and it's run locally.

Also, in order to run this notebook, it is required to add your CDSAPI token inside the `keys.json` file provided inside the repository.

### Reference material:<br> 
- GitHub Repository containing examples from ECMWF: https://github.com/ecmwf-projects/copernicus-training
- Example: https://github.com/ecmwf-projects/copernicus-training/blob/master/2021-02-Copernicus-ECMWF-data-tutorial.ipynb

### Import libraries and API authentication

In [5]:
import warnings
warnings.filterwarnings('ignore')

import cdsapi
import os
import geopandas as gpd
import xarray as xr
import json
import ipywidgets as widgets
import ee
import geemap
import datetime


# Current working directory path:
cwd = os.getcwd()

# Import functions defined for DDUST project:
from functions import DDUST_methods

In [2]:
# Google Earth Engine authentication (a Google account is required):
ee.Authenticate()

Enter verification code:  4/1AdQt8qi6FiacQBRNQfaraVSPfjnvj8HWFsESCSTh6b6njULGxCekS9YYxSs



Successfully saved authorization token.


In [3]:
# Initialize Google Earth Engine API:
ee.Initialize()

In [6]:
# CDSAPI: Importing key/value from .json file for API authentication:
f = open('keys.json')
keys = json.load(f)
c_atm = cdsapi.Client(keys["ATM_ID"], keys["ATM_KEY"])

### Set starting and end date for data request:

The date selection is done through the `date.json` file that contains the date range for requesting the data. The `date.json` file is the same for all the project's notebooks:

In [7]:
d = open('date.json') #Open .json file where manuring weeks and year are saved
date = json.load(d)
year = date['year']
custom_week = date['custom_week']

In [8]:
# Select start and end date of the corresponding selected week:
start_date = str(datetime.datetime.strptime((str(year)+'-'+custom_week[0]), "%Y-%m-%d").date())
end_date = str(datetime.datetime.strptime((str(year)+'-'+custom_week[1]), "%Y-%m-%d").date())
print("The starting date is", start_date,"and the ending date is" , end_date,". The date is define as yyyy-mm-dd.")

The starting date is 2021-03-10 and the ending date is 2021-03-15 . The date is define as yyyy-mm-dd.


### Import Bounding Box

In [9]:
area_path = cwd + '/grid/grid_0_1.gpkg'
area = gpd.read_file(area_path).to_crs(4326)
bounds = area.total_bounds
roi = ee.Geometry.BBox(bounds[0],bounds[1],bounds[2],bounds[3])

Define the scale for the ERA5 data download:

In [10]:
era5_scale = 1000

- - -

# CDAPI Data Requests for Copernicus Atmosphere Monitoring Service (CAMS)

This part contains the requests for the following variables:
- Particulate matter (PM10, PM2.5)
- Ammonia (NH3)
- Sulphur Dioxide (SO2)
- Nitrogen Dioxide (NO2)
- Nitrogen Monoxide (NO)
- Carbon Monoxide (CO)
- Ozone (O3)
- Dust
- NMVOCs (Non-Methane Volatile Organic Compounds)

Each section just requires hourly data in the time range (start & end date) for each variable. The data are required in the bounding box only, and a mean value it's calculated. The output format is a netCDF file.

# [PM2.5 - Particulate Matter](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) 

In [11]:
pm25_path = cwd+r'\temp\pm25_cams.nc'
pm25 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'particulate_matter_2.5um',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    pm25_path
    )

2022-08-22 10:58:45,527 INFO Welcome to the CDS
2022-08-22 10:58:45,527 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-08-22 10:58:45,564 INFO Request is queued
2022-08-22 10:58:46,597 INFO Request is running
2022-08-22 10:59:18,012 INFO Request is completed
2022-08-22 10:59:18,012 INFO Downloading https://download-0000-ads-clone.copernicus-climate.eu/cache-compute-0000/cache/data8/adaptor.cams_regional_fc.retrieve-1661158747.0440676-25494-14-9a4d1a51-e72b-4349-8351-b6de67e0c797.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\pm25_cams.nc (368.2K)
2022-08-22 10:59:18,331 INFO Download rate 1.1M/s


In [12]:
pm25 = xr.open_dataset(pm25_path)

In [13]:
pm25_mean = pm25.mean(dim='time')

In [14]:
pm25_mean.to_netcdf(pm25_path)

- - -

# [NH3 - Ammonia](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) 

In [15]:
nh3_path = cwd+r'\temp\nh3_cams.nc'
nh3 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'ammonia',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    nh3_path
    )

2022-08-22 10:59:18,785 INFO Welcome to the CDS
2022-08-22 10:59:18,786 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-08-22 10:59:18,823 INFO Request is queued
2022-08-22 10:59:19,867 INFO Request is running
2022-08-22 10:59:51,270 INFO Request is completed
2022-08-22 10:59:51,272 INFO Downloading https://download-0005-ads-clone.copernicus-climate.eu/cache-compute-0005/cache/data9/adaptor.cams_regional_fc.retrieve-1661158781.424463-792-11-3d9ecf87-f8f2-482e-82d2-7a5c7e2065a1.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\nh3_cams.nc (368.2K)
2022-08-22 10:59:51,513 INFO Download rate 1.5M/s


In [16]:
nh3 = xr.open_dataset(nh3_path)

In [17]:
nh3_mean = nh3.mean(dim='time')

In [18]:
nh3_mean.to_netcdf(nh3_path)

- - -

# [SO2 - Sulphur Dioxide](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) 

In [19]:
so2_path = cwd + r'/temp/so2_cams.nc'
so2 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'sulphur_dioxide',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    so2_path
    )

2022-08-22 10:59:51,661 INFO Welcome to the CDS
2022-08-22 10:59:51,662 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-08-22 10:59:51,700 INFO Request is queued
2022-08-22 10:59:52,735 INFO Request is running
2022-08-22 11:00:24,150 INFO Request is completed
2022-08-22 11:00:24,150 INFO Downloading https://download-0002-ads-clone.copernicus-climate.eu/cache-compute-0002/cache/data9/adaptor.cams_regional_fc.retrieve-1661158813.226386-25081-3-ced715c3-4a36-42a4-a46a-07ab13c2ca31.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/so2_cams.nc (368.2K)
2022-08-22 11:00:24,407 INFO Download rate 1.4M/s


In [20]:
so2 = xr.open_dataset(so2_path)

In [21]:
so2_mean = so2.mean(dim='time')

In [22]:
so2_mean.to_netcdf(so2_path)

- - -

# [NO2 - Nitrogen Dioxide](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) 

In [23]:
no2_path = cwd + r'/temp/no2_cams.nc'
no2 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'nitrogen_dioxide',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    no2_path
    )

2022-08-22 11:00:24,543 INFO Welcome to the CDS
2022-08-22 11:00:24,544 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-08-22 11:00:24,580 INFO Request is queued
2022-08-22 11:00:25,614 INFO Request is running
2022-08-22 11:00:57,020 INFO Request is completed
2022-08-22 11:00:57,021 INFO Downloading https://download-0004-ads-clone.copernicus-climate.eu/cache-compute-0004/cache/data7/adaptor.cams_regional_fc.retrieve-1661158847.6296396-9994-14-2b8b411f-fc62-4372-a395-6d9c771a6bd1.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/no2_cams.nc (368.2K)
2022-08-22 11:00:57,283 INFO Download rate 1.4M/s 


In [24]:
no2 = xr.open_dataset(no2_path)

In [25]:
no2_mean = no2.mean(dim='time')

In [26]:
no2_mean.to_netcdf(no2_path)

- - -

# [NO - Nitrogen Monoxide](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

In [27]:
no_path = cwd + r'/temp/no_cams.nc'
no = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'nitrogen_monoxide',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    no_path
    )

2022-08-22 11:00:57,421 INFO Welcome to the CDS
2022-08-22 11:00:57,422 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-08-22 11:00:57,459 INFO Request is queued
2022-08-22 11:00:58,489 INFO Request is running
2022-08-22 11:01:29,894 INFO Request is completed
2022-08-22 11:01:29,894 INFO Downloading https://download-0005-ads-clone.copernicus-climate.eu/cache-compute-0005/cache/data8/adaptor.cams_regional_fc.retrieve-1661158879.3712132-30265-10-b20b5bf6-32b7-477b-8b97-f70a4128e8bb.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/no_cams.nc (368.2K)
2022-08-22 11:01:30,109 INFO Download rate 1.7M/s 


In [28]:
no = xr.open_dataset(no_path)

In [29]:
no_mean = no.mean(dim='time')

In [30]:
no_mean.to_netcdf(no_path)

- - -

# [CO - Carbon Monoxide](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

In [31]:
co_path = cwd + r'/temp/co_cams.nc'
co = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'carbon_monoxide',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    co_path
    )

2022-08-22 11:01:30,240 INFO Welcome to the CDS
2022-08-22 11:01:30,241 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-08-22 11:01:30,277 INFO Request is queued
2022-08-22 11:01:31,307 INFO Request is running
2022-08-22 11:02:02,801 INFO Request is completed
2022-08-22 11:02:02,802 INFO Downloading https://download-0005-ads-clone.copernicus-climate.eu/cache-compute-0005/cache/data0/adaptor.cams_regional_fc.retrieve-1661158912.7619529-8502-14-9788ea9a-08c4-4188-97fc-c6c1e711bf5e.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/co_cams.nc (368.2K)
2022-08-22 11:02:03,344 INFO Download rate 680.8K/s


In [32]:
co = xr.open_dataset(co_path)

In [33]:
co_mean = co.mean(dim='time')

In [34]:
co_mean.to_netcdf(co_path)

---

# [O3 - Ozone](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

In [35]:
o3_path = cwd + r'/temp/o3_cams.nc'
o3 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'ozone',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    o3_path
    )

2022-08-22 11:02:03,489 INFO Welcome to the CDS
2022-08-22 11:02:03,490 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-08-22 11:02:03,530 INFO Request is queued
2022-08-22 11:02:04,566 INFO Request is running
2022-08-22 11:02:35,975 INFO Request is completed
2022-08-22 11:02:35,976 INFO Downloading https://download-0001-ads-clone.copernicus-climate.eu/cache-compute-0001/cache/data3/adaptor.cams_regional_fc.retrieve-1661158945.0054576-29562-4-92b14a14-5f08-40f8-a024-948dee2124d3.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/o3_cams.nc (368.1K)
2022-08-22 11:02:36,234 INFO Download rate 1.4M/s


In [36]:
o3 = xr.open_dataset(o3_path)

In [37]:
o3_mean = o3.mean(dim='time')

In [38]:
o3_mean.to_netcdf(o3_path)

---

# [Dust](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

In [39]:
dust_path = cwd + r'/temp/dust_cams.nc'
dust = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'dust',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    dust_path
    )

2022-08-22 11:02:36,381 INFO Welcome to the CDS
2022-08-22 11:02:36,381 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-08-22 11:02:36,418 INFO Request is queued
2022-08-22 11:02:37,461 INFO Request is running
2022-08-22 11:02:57,434 INFO Request is completed
2022-08-22 11:02:57,435 INFO Downloading https://download-0001-ads-clone.copernicus-climate.eu/cache-compute-0001/cache/data8/adaptor.cams_regional_fc.retrieve-1661158977.204503-30446-14-c0d0e20c-8713-4c5d-b3fa-a196c2ec4d8e.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/dust_cams.nc (368.1K)
2022-08-22 11:02:57,594 INFO Download rate 2.3M/s 


In [40]:
dust = xr.open_dataset(dust_path)

In [41]:
dust_mean = dust.mean(dim='time')

In [42]:
dust_mean.to_netcdf(dust_path)

# [PM10 - Particulate Matter](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

In [43]:
pm10_path = cwd + r'/temp/pm10_cams.nc'
pm10 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'model': 'ensemble',
        'date': start_date+'/'+end_date,
        'format': 'netcdf',
        'level': '0',
        'type': 'analysis',
        'variable': 'particulate_matter_10um',
        'leadtime_hour': '0',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2]],
        'format': 'netcdf'
    },
    pm10_path)

2022-08-22 11:02:57,738 INFO Welcome to the CDS
2022-08-22 11:02:57,739 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-08-22 11:02:57,774 INFO Request is queued
2022-08-22 11:02:58,820 INFO Request is running
2022-08-22 11:03:18,791 INFO Request is completed
2022-08-22 11:03:18,791 INFO Downloading https://download-0001-ads-clone.copernicus-climate.eu/cache-compute-0001/cache/data0/adaptor.cams_regional_fc.retrieve-1661158998.1837306-30049-12-86309ef1-ee44-4c9b-9b3d-e92a16a00eb0.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/pm10_cams.nc (368.2K)
2022-08-22 11:03:18,937 INFO Download rate 2.5M/s 


In [44]:
pm10 = xr.open_dataset(pm10_path)

In [45]:
pm10_mean = pm10.mean(dim='time')

In [46]:
pm10_mean.to_netcdf(pm10_path)

# [NMVOCs - Non Methane VOCs](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

In [47]:
nmvocs_path = cwd + r'/temp/nmvocs_cams.nc'
nmvocs = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'model': 'ensemble',
        'date': start_date+'/'+end_date,
        'format': 'netcdf',
        'level': '0',
        'type': 'analysis',
        'variable': 'non_methane_vocs',
        'leadtime_hour': '0',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2]],
        'format': 'netcdf'
    },
    nmvocs_path)

2022-08-22 11:03:19,078 INFO Welcome to the CDS
2022-08-22 11:03:19,079 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-08-22 11:03:19,117 INFO Request is queued
2022-08-22 11:03:20,162 INFO Request is running
2022-08-22 11:03:51,566 INFO Request is completed
2022-08-22 11:03:51,567 INFO Downloading https://download-0004-ads-clone.copernicus-climate.eu/cache-compute-0004/cache/data7/adaptor.cams_regional_fc.retrieve-1661159020.2213578-12004-14-c0a35a7f-d313-4b87-9866-ed8acd5fe2be.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/nmvocs_cams.nc (368.2K)
2022-08-22 11:03:51,804 INFO Download rate 1.5M/s 


In [48]:
nmvocs = xr.open_dataset(nmvocs_path)

In [49]:
nmvocs_mean = nmvocs.mean(dim='time')

In [50]:
nmvocs_mean.to_netcdf(nmvocs_path)

---

# [ECMWF C3S ERA-5 Climate Reanalysis](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_HOURLY) 
WCMWF C3S ERA-5 Model data are retrieved using Google Earth Engine API.
The data selected are:
 - Air temperature at 2 m above the land surface
 - Precipitation is defined as accumulated liquid and frozen water, including rain and snow
 - Pressure is defined as the weight of all the air in a column vertically above the area of the Earth's surface represented at a fixed point.
 - Northward and eastward wind (u_wind, v_component 10 m above the land surface
 - Soil humidity is defined as volume of water in soil layer 1 (0 - 7 cm)

In [51]:
Map_era5 = geemap.Map(center=[45.5,10], zoom = 7 )
Map_era5

Map(center=[45.5, 10], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [52]:
# Daily mean 2m air temperature
temp_2m = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')\
                   .select('temperature_2m')\
                   .filter(ee.Filter.date(start_date, end_date));

# Daily total precipitation sums
prec = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')\
                  .select('total_precipitation')\
                  .filter(ee.Filter.date(start_date, end_date));

# Daily mean surface pressure
press = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')\
                  .select('surface_pressure')\
                  .filter(ee.Filter.date(start_date, end_date));

# Daily mean 10m u-component of wind
u_wind = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')\
                          .select('u_component_of_wind_10m')\
                          .filter(ee.Filter.date(start_date, end_date));

# Daily mean 10m v-component of wind
v_wind = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')\
                          .select('v_component_of_wind_10m')\
                          .filter(ee.Filter.date(start_date, end_date));


# Daily volumetric soil water content (level 1)
hum = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')\
                          .select('volumetric_soil_water_layer_1')\
                          .filter(ee.Filter.date(start_date, end_date));


# Visualization palette for temperature at 2m
temp_2m_viz  = {
  'min': 220,
  'max': 304,
  'palette': ['#FFFFFF', '#00FFFF', '#0080FF', '#DA00FF', '#FFA400', '#FF0000']
};

# Visualization palette for pressure (surface pressure)
press_viz = {
  'min': 65000,
  'max': 120000,
  'palette': [
    '#01FFFF', '#058BFF', '#0600FF', '#DF00FF', '#FF00FF', '#FF8C00', '#FF8C00'
  ]
};

# Visualization palette for total precipitation
prec_viz  = {
  'min': 0,
  'max': 0.1,
  'palette': ['#FFFFFF', '#00FFFF', '#0080FF', '#DA00FF', '#FFA400', '#FF0000']
};

# Visualization palette for u-component of 10m wind
u_wind_viz = {
  'min': 0,
  'max': 30,
  'palette': [
    '#FFFFFF', '#FFFF71', '#DEFF00', '#9EFF00', '#77B038', '#007E55', '#005F51',
    '#004B51', '#013A7B', '#023AAD'
  ]
}
                           
# Visualization palette for v-component of 10m wind
v_wind_viz = {
  'min': 0,
  'max': 30,
  'palette': [
    '#FFFFFF', '#FFFF71', '#DEFF00', '#9EFF00', '#77B038', '#007E55', '#005F51',
    '#004B51', '#013A7B', '#023AAD'
  ]
};

# Visualization palette for soil humitidy
hum_viz = {
  'min': 0,
  'max': 1,
  'palette': [
    '#FFFFFF', '#FFFF71', '#DEFF00', '#9EFF00', '#77B038', '#007E55', '#005F51',
    '#004B51', '#013A7B', '#023AAD'
  ]
};

temp_2m = temp_2m.mean()
press = press.mean()
prec = prec.mean()
u_wind = u_wind.mean()
v_wind = v_wind.mean()
hum = hum.mean()

In [53]:
#Clip over the ROI previously defined
temp_2m_clip = temp_2m.clip(roi)
press_clip = press.clip(roi)
prec_clip = prec.clip(roi)
u_wind_clip = u_wind.clip(roi)
v_wind_clip = v_wind.clip(roi)
hum_clip = hum.clip(roi)

Map_era5.addLayer(temp_2m_clip, temp_2m_viz, '2m temperature');
Map_era5.addLayer(press_clip, press_viz, 'press');
Map_era5.addLayer(prec_clip, prec_viz, 'precipitation');
Map_era5.addLayer(u_wind_clip, u_wind_viz, 'u wind');
Map_era5.addLayer(v_wind_clip, v_wind_viz, 'v wind');
Map_era5.addLayer(hum_clip, hum_viz, 'soil humitidy');

In [54]:
out_dir = os.path.expanduser(cwd + '/temp')
filename = os.path.join(out_dir, 'temp_2m.tif')
geemap.ee_export_image(temp_2m_clip, filename=filename, scale=era5_scale)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\temp_2m.tif


In [55]:
out_dir = os.path.expanduser(cwd + '/temp')
filename = os.path.join(out_dir, 'press.tif')
geemap.ee_export_image(press_clip, filename=filename, scale=era5_scale)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\press.tif


In [56]:
out_dir = os.path.expanduser(cwd + '/temp')
filename = os.path.join(out_dir, 'prec.tif')
geemap.ee_export_image(prec_clip, filename=filename, scale=era5_scale)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\prec.tif


In [57]:
out_dir = os.path.expanduser(cwd + '/temp')
filename = os.path.join(out_dir, 'e_wind.tif')
geemap.ee_export_image(u_wind_clip, filename=filename, scale=era5_scale)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\e_wind.tif


In [58]:
out_dir = os.path.expanduser(cwd + '/temp')
filename = os.path.join(out_dir, 'n_wind.tif')
geemap.ee_export_image(v_wind_clip, filename=filename, scale=era5_scale)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\n_wind.tif


In [59]:
out_dir = os.path.expanduser(cwd + '/temp')
filename = os.path.join(out_dir, 'soil_hum.tif')
geemap.ee_export_image(hum_clip, filename=filename, scale=era5_scale)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\soil_hum.tif
